In [25]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
'''
@File    :   test_higashi.ipynb
@Time    :   2024/01/04 17:12:03
@Author  :   cp 
@Version :   1.0
@Desc    :   None
'''
import sys
sys.path.append('/home/higashi/Higashi/higashi')


from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse



def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	#生成一个稀疏矩阵尺寸的全为0的矩阵
	a = np.zeros((size, size))
	#默认调用时传递k = 100000
	#如果该染色体被划分为小于10万个bin 则相当于还是全0矩阵
	#
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a

	# print(a)
	# 对bulk矩阵每一行求和，如果对应行的和等于0 则为true 否则为false
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	print(gap)
	# 如果cytoband文件存在
	if cytoband_path is not None:
		#读取该文件为表格
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		#定义列名
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		#获取name列
		name = np.array(gap_tab['name'])
		# print (name)
		#qC1.1
		#获取name 第一个字符
		pqarm = np.array([str(s)[0] for s in name])
		#作为gap_tab的新列
		gap_tab['pq_arm'] = pqarm
		#获取 区域长度作为新列
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		#按照 'chrom' 和 'pq_arm' 分组并对数值列进行求和
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		#如果求和结果中['pq_arm'] == 'p'的和大于0
		#则划分点为该染色体中 对符合条件（summarize['pq_arm'] == 'p')）的 'length' 列数据每个元素除以 分辨率 并向上取整后的第一个元素
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		#否则 置划分点为-1
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	#如果cytoband文件不存在
	#直接置split_point = -1
	else:
		split_point = -1
	#将矩阵a 中gap为true的对应位置置1，如：gap第一个位置为true。则a的第一行 第一列置为1
	a[gap, :] = 1
	a[:, gap] = 1
	#返回结果a 和转换为整数的划分点
	return a, int(split_point)


def process_one_chrom(chrom ,path):
	# Get the raw sparse mtx list
	# origin_sparse = np.load(os.path.join('/home/dataset/snm3c/250k/raw', "%s_sparse_matrices.npy" % chrom), allow_pickle=True)
	origin_sparse = np.load(path, allow_pickle=True)
	size = origin_sparse[0].shape[0]
	print('size',size)
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)
	print('split_point',split_point)
	
	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print('bulk1.shape',bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask
	
	
	bulk2 = None
	
	use_rows_all = []
	
	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]
	
	bulk_compartment_all = []
	real_bulk_compartment_all = []
	
	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	temp_compartment_list_zscore = []
	temp_compartment_list_quantile = []
	
	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		print('len use_rows',len(use_rows))



chrom = 'chr1'
# cytoband_path ='/home/dataset/snm3c/250k/cytoBand.txt'
cytoband_path ='/home/test_tmp/mm9cytoBand.txt'
# npy_path =os.path.join('/home/dataset/snm3c/250k/raw', "%s_sparse_matrices.npy" % chrom)
npy_path = '/home/test_tmp/chr1.npy'
res = 250000

process_one_chrom(chrom,npy_path)


size 789
[ True  True  True  True  True  True  True  True  True  True  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False

In [30]:
str('qc1.1')[0]

'q'

In [38]:
import pandas as pd
def split_test(origin_sparse):
    # final = np.array(np.sum(origin_sparse, axis=0).todense())
    # size = origin_sparse[0].shape[-1]
    # #生成一个稀疏矩阵尺寸的全为0的矩阵
    # a = np.zeros((size, size))
    # #默认调用时传递k = 100000
    # #如果该染色体被划分为小于10万个bin 则相当于还是全0矩阵
    # #
    # k = int(1e5)
    # if k > 0:
    #     for i in range(min(k, len(a))):
    #         for j in range(len(a) - i):
    #             a[j, j + i] = 1
    #             a[j + i, j] = 1
    #     a = np.ones_like((a)) - a

    # # print(a)
    # # 对bulk矩阵每一行求和，如果对应行的和等于0 则为true 否则为false
    # gap = np.sum(final, axis=-1, keepdims=False) == 0
    # print(gap)
    # 如果cytoband文件存在
    if cytoband_path is not None:
        #读取该文件为表格
        gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
        #定义列名
        gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
        #获取name列
        gap_tab.head()
        name = np.array(gap_tab['name'])
        # print (name)
        #qC1.1
        #获取name 第一个字符
        # p为短臂，q为长臂
        pqarm = np.array([str(s)[0] for s in name])
        #作为gap_tab的新列
        gap_tab['pq_arm'] = pqarm
        #获取 区域长度作为新列 即该条带长度
        gap_tab['length'] = gap_tab['end'] - gap_tab['start']
        gap_tab.head()
        #按照 'chrom' 和 'pq_arm' 分组并对数值列进行求和
        #实际'length'和为染色体长度
        summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
        # print (summarize)
        #如果求和结果中['pq_arm'] == 'p'的和大于0
        #对在短臂的条带处理
        #则划分点为该染色体中 对符合条件（summarize['pq_arm'] == 'p')）的 'length' 列数据每个元素除以 分辨率 并向上取整后的第一个元素
        if np.sum(summarize['pq_arm'] == 'p') > 0:
            split_point = \
            np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
        #否则 置划分点为-1
        else:
            split_point = -1

        #acen是着色粒
        #选出着色粒所在区域
        gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
        #np.floor返回输入每个元素的下限
        start = np.floor((np.array(gap_list['start'])) / res).astype('int')
        #np.ceil向上取整 
        end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
        #将所在区域对应的bin的区域在矩阵a中置1
        
        # for s, e in zip(start, end):
        #     a[s:e, :] = 1
        #     a[:, s:e] = 1
    #如果cytoband文件不存在
    #直接置split_point = -1
    else:
        split_point = -1
    #将矩阵a 中gap为true的对应位置置1，如：gap第一个位置为true。则a的第一行 第一列置为1
    # a[gap, :] = 1
    # a[:, gap] = 1
    #返回结果a 和转换为整数的划分点
    return int(split_point)

chrom = 'chr1'
cytoband_path ='/home/dataset/snm3c/250k/cytoBand.txt'
# cytoband_path ='/home/test_tmp/mm9cytoBand.txt'
# npy_path =os.path.join('/home/dataset/snm3c/250k/raw', "%s_sparse_matrices.npy" % chrom)
npy_path = '/home/test_tmp/chr1.npy'
res = 250000
origin_sparse = np.load(npy_path, allow_pickle=True)
split_test(origin_sparse)





-1

In [44]:
cytoband_path = '/home/annotation/hg19/cytoBand.txt'
gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
        #定义列名
gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
#获取name列
gap_tab.head()

,chrom,start,end,name,type
0,chr1,0,2300000,p36.33,gneg
1,chr1,2300000,5400000,p36.32,gpos25
2,chr1,5400000,7200000,p36.31,gneg
3,chr1,7200000,9200000,p36.23,gpos25
4,chr1,9200000,12700000,p36.22,gneg


In [45]:
name = np.array(gap_tab['name'])
# print (name)
#qC1.1
#获取name 第一个字符
pqarm = np.array([str(s)[0] for s in name])
#作为gap_tab的新列
gap_tab['pq_arm'] = pqarm
#获取 区域长度作为新列
gap_tab['length'] = gap_tab['end'] - gap_tab['start']
gap_tab.head()

,chrom,start,end,name,type,pq_arm,length
0,chr1,0,2300000,p36.33,gneg,p,2300000
1,chr1,2300000,5400000,p36.32,gpos25,p,3100000
2,chr1,5400000,7200000,p36.31,gneg,p,1800000
3,chr1,7200000,9200000,p36.23,gpos25,p,2000000
4,chr1,9200000,12700000,p36.22,gneg,p,3500000


In [46]:

summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
summarize
#249250621



,chrom,pq_arm,start,end,name,type,length
0,chr1,p,1976100000,2101100000,p36.33p36.32p36.31p36.23p36.22p36.21p36.13p36....,gneggpos25gneggpos25gneggpos50gneggpos25gneggp...,125000000
1,chr1,q,5381400000,5505650621,q11q12q21.1q21.2q21.3q22q23.1q23.2q23.3q24.1q2...,acengvargneggpos50gneggpos50gneggpos50gneggpos...,124250621
2,chr10,p,260700000,300900000,p15.3p15.2p15.1p14p13p12.33p12.32p12.31p12.2p1...,gneggpos25gneggpos75gneggpos75gneggpos75gneggp...,40200000
3,chr10,q,2487400000,2582734747,q11.1q11.21q11.22q11.23q21.1q21.2q21.3q22.1q22...,acengneggpos25gneggpos100gneggpos100gneggpos50...,95334747
4,chr11,p,328700000,382400000,p15.5p15.4p15.3p15.2p15.1p14.3p14.2p14.1p13p12...,gneggpos50gneggpos50gneggpos100gneggpos75gnegg...,53700000
5,chr11,q,2061400000,2142706516,q11q12.1q12.2q12.3q13.1q13.2q13.3q13.4q13.5q14...,acengpos75gneggpos25gneggpos25gneggpos50gneggp...,81306516
6,chr12,p,206000000,241800000,p13.33p13.32p13.31p13.2p13.1p12.3p12.2p12.1p11...,gneggpos25gneggpos75gneggpos100gneggpos100gneg...,35800000
7,chr12,q,2442000000,2540051895,q11q12q13.11q13.12q13.13q13.2q13.3q14.1q14.2q1...,acengpos100gneggpos25gneggpos25gneggpos75gnegg...,98051895
8,chr13,p,30800000,48700000,p13p12p11.2p11.1,gvarstalkgvaracen,17900000
9,chr13,q,1984300000,2081569878,q11q12.11q12.12q12.13q12.2q12.3q13.1q13.2q13.3...,acengneggpos25gneggpos25gneggpos50gneggpos75gn...,97269878


In [31]:
a = np.array([
    [0, 0, 0, 0, 0],
    [5, 2, 8, 1, 7],
    [9, 2, 6, 5, 3],
    [1, 4, 7, 2, 8],
    [3, 6, 0, 9, 5]
])
b=np.sum(a, axis=-1, keepdims=False)==0
b

array([ True, False, False, False, False])

In [35]:
c = np.zeros((5, 5))
c[b,:]=1
c[:,b]=1
c

array([[1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [15]:
a = np.zeros((10, 10))

for i in range(len(a)):
    for j in range(len(a) - i):
        a[j, j + i] = 1
        a[j + i, j] = 1
# a = np.ones_like((a)) - a
a = np.ones_like(a) - a

a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [23]:
def create_mask_test(k=30):
    print(k)
create_mask_test((int(1e5)))

100000


In [22]:
import numpy as np

a = np.zeros((10, 10))

for i in range(len(a)):
    for j in range(len(a) - i):
        print(j,j+i)
        a[j, j + i] = 1
        a[j + i, j] = 1

# a = np.ones_like(a)-a
a

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
0 2
1 3
2 4
3 5
4 6
5 7
6 8
7 9
0 3
1 4
2 5
3 6
4 7
5 8
6 9
0 4
1 5
2 6
3 7
4 8
5 9
0 5
1 6
2 7
3 8
4 9
0 6
1 7
2 8
3 9
0 7
1 8
2 9
0 8
1 9
0 9


array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])